## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Atar,20.5169,-13.0499,72.00,29,62,5.86,MR,broken clouds
1,1,Chokurdakh,70.6333,147.9167,-14.62,94,88,6.46,RU,overcast clouds
2,2,Champerico,14.3000,-91.9167,80.35,80,69,1.34,GT,broken clouds
3,3,Busselton,-33.6500,115.3333,69.04,48,3,10.04,AU,clear sky
4,4,Socorro,31.6546,-106.3033,71.89,27,20,6.91,US,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 0
What is your desired maximum temperature for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Atar,20.5169,-13.0499,72.00,29,62,5.86,MR,broken clouds
2,2,Champerico,14.3000,-91.9167,80.35,80,69,1.34,GT,broken clouds
3,3,Busselton,-33.6500,115.3333,69.04,48,3,10.04,AU,clear sky
4,4,Socorro,31.6546,-106.3033,71.89,27,20,6.91,US,few clouds
5,5,Labuhan,-6.8844,112.2051,87.28,61,100,3.69,ID,overcast clouds
6,6,Manggar,-2.8833,108.2667,86.16,65,99,3.58,ID,overcast clouds
7,7,Mataura,-46.1927,168.8643,72.90,40,100,1.14,NZ,overcast clouds
8,8,Moba,33.2288,105.0179,27.03,81,49,3.29,CN,scattered clouds
9,9,Aswan,24.0934,32.9070,74.70,34,0,8.16,EG,clear sky
10,10,Nara,34.6851,135.8049,72.50,43,93,15.48,JP,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                4
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Atar,20.5169,-13.0499,72.00,29,62,5.86,MR,broken clouds
2,2,Champerico,14.3000,-91.9167,80.35,80,69,1.34,GT,broken clouds
3,3,Busselton,-33.6500,115.3333,69.04,48,3,10.04,AU,clear sky
4,4,Socorro,31.6546,-106.3033,71.89,27,20,6.91,US,few clouds
5,5,Labuhan,-6.8844,112.2051,87.28,61,100,3.69,ID,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
714,714,Anadyr,64.7500,177.4833,12.70,100,40,6.71,RU,fog
715,715,Payo,-3.7508,103.6372,76.19,95,100,1.41,ID,overcast clouds
716,716,Lakselv,70.0513,24.9718,30.74,93,75,10.36,NO,light snow
717,717,Bocaiuva,-17.1078,-43.8150,73.11,91,77,3.09,BR,light rain


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atar,MR,72.00,broken clouds,20.5169,-13.0499,
2,Champerico,GT,80.35,broken clouds,14.3000,-91.9167,
3,Busselton,AU,69.04,clear sky,-33.6500,115.3333,
4,Socorro,US,71.89,few clouds,31.6546,-106.3033,
5,Labuhan,ID,87.28,overcast clouds,-6.8844,112.2051,
6,Manggar,ID,86.16,overcast clouds,-2.8833,108.2667,
7,Mataura,NZ,72.90,overcast clouds,-46.1927,168.8643,
8,Moba,CN,27.03,scattered clouds,33.2288,105.0179,
9,Aswan,EG,74.70,clear sky,24.0934,32.9070,
10,Nara,JP,72.50,overcast clouds,34.6851,135.8049,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Sk

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atar,MR,72.00,broken clouds,20.5169,-13.0499,Oumou Elghoura
2,Champerico,GT,80.35,broken clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
3,Busselton,AU,69.04,clear sky,-33.6500,115.3333,Observatory Guest House
4,Socorro,US,71.89,few clouds,31.6546,-106.3033,guarderia imss papalote
5,Labuhan,ID,87.28,overcast clouds,-6.8844,112.2051,PT PPI Lamongan Tambak C


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))